# Create electricity prices

Day-ahead Price [EUR/MWh], converted to [EUR/kWh]

In [1]:
%%capture
!pip install plotly==5.9.0

In [2]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/data/data_validating/
%ls

In [4]:
#Moving to the current directory
current_folder = ''

input_folder = current_folder + 'raw_datasets/'
output_folder = current_folder + 'scenario_datasets/'

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px

In [6]:
prices_data = pd.read_csv(input_folder + 'Day-ahead Prices_201801010000-201901010000.csv')

#prices_data.head()
prices_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   MTU (CET/CEST)             8761 non-null   object 
 1   Day-ahead Price [EUR/MWh]  8758 non-null   object 
 2   Currency                   6553 non-null   object 
 3   BZN|DE-AT-LU               0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 273.9+ KB


In [7]:
prices_data["MTU (CET/CEST)"] = prices_data["MTU (CET/CEST)"].str.slice(start=0, stop=16)
prices_data.rename(columns = {'MTU (CET/CEST)':'time'}, inplace = True)
prices_data['time'] = pd.to_datetime(prices_data['time'])

#COnversion from EUR/MWh to EUR/kWh
prices_data.rename(columns = {'Day-ahead Price [EUR/MWh]':'EUR/kWh'}, inplace = True)
prices_data['EUR/kWh'] = pd.to_numeric(prices_data['EUR/kWh'],errors='coerce') / 1000

prices_data = prices_data.drop(['BZN|DE-AT-LU'], axis=1)
prices_data = prices_data.drop(['Currency'], axis=1)

index = len(prices_data['time'])
timestamp = pd.to_datetime("2019-01-01 00:00:00", format="%Y-%m-%d %H:%M:%S")
last_value = prices_data['EUR/kWh'][index-1]
new_row = pd.DataFrame([[timestamp, last_value]], columns=["time",'EUR/kWh'], index=[index])
prices_data = pd.concat([prices_data, pd.DataFrame(new_row)], ignore_index=False)

prices_data.tail()

,time,EUR/kWh
8757,2018-12-31 20:00:00,0.03294
8758,2018-12-31 21:00:00,0.03315
8759,2018-12-31 22:00:00,0.03350
8760,2018-12-31 23:00:00,0.03827
8761,2019-01-01 00:00:00,0.03827


Data resampling

In [8]:
prices_data = prices_data.resample('15min', on='time').agg({'time':'min','EUR/kWh':'min'})

#prices_data['time'] = prices_data.index.values
prices_data['EUR/kWh'].fillna(method='ffill', inplace=True)
prices_data = prices_data.loc['2018-01-01 00:00:00':'2019-01-01 00:00:00']
prices_data = prices_data.drop(['time'], axis=1)
prices_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35041 entries, 2018-01-01 00:00:00 to 2019-01-01 00:00:00
Freq: 15T
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   EUR/kWh  35041 non-null  float64
dtypes: float64(1)
memory usage: 547.5 KB


Saving and plotting dataset

In [9]:
prices_csv = prices_data.to_csv(output_folder + 'market_prices_2018_profile.csv', index = True)
px.line(prices_data['EUR/kWh'], title=('Electricity prices')).update_layout(yaxis_title="€/kWh", xaxis_rangeslider_visible=True, xaxis_range=["2018-06-01 00:00:00", "2018-07-01 00:00:00"])